# Example: 在Dynex平台上使用Python进行计算 - NAE3SAT

Dynex是全球首个基于DynexSolve芯片算法的神经形态超级计算区块链，采用了一种“有用工作证明”（PoUW）方法来解决现实世界问题。此示例演示了如何使用Dynex SDK在Dynex平台上使用Python进行计算。

In [1]:
import dynex
import dimod

## Building a Binary Quadratic Model 

构建一个二次二元模型

<math xmlns="http://www.w3.org/1998/Math/MathML" display="block">
  <mi>E</mi>
  <mo stretchy="false">(</mo>
  <mrow data-mjx-texclass="ORD">
    <mi mathvariant="bold">v</mi>
  </mrow>
  <mo mathvariant="bold" stretchy="false">)</mo>
  <mo mathvariant="bold">=</mo>
  <munder>
    <mo data-mjx-texclass="OP">&#x2211;</mo>
    <mrow data-mjx-texclass="ORD">
      <mi mathvariant="bold">i</mi>
    </mrow>
  </munder>
  <msub>
    <mi mathvariant="bold">a</mi>
    <mi mathvariant="bold">i</mi>
  </msub>
  <msub>
    <mi mathvariant="bold">v</mi>
    <mi mathvariant="bold">i</mi>
  </msub>
  <mo mathvariant="bold">+</mo>
  <munder>
    <mo data-mjx-texclass="OP">&#x2211;</mo>
    <mrow data-mjx-texclass="ORD">
      <mi mathvariant="bold">i</mi>
      <mo mathvariant="bold">&lt;</mo>
      <mi mathvariant="bold">j</mi>
    </mrow>
  </munder>
  <msub>
    <mi mathvariant="bold">b</mi>
    <mrow data-mjx-texclass="ORD">
      <mi mathvariant="bold">i</mi>
      <mo mathvariant="bold">,</mo>
      <mi mathvariant="bold">j</mi>
    </mrow>
  </msub>
  <msub>
    <mi mathvariant="bold">v</mi>
    <mi mathvariant="bold">i</mi>
  </msub>
  <msub>
    <mi mathvariant="bold">v</mi>
    <mi mathvariant="bold">j</mi>
  </msub>
  <mo mathvariant="bold">+</mo>
  <mi mathvariant="bold">c</mi>
  <mstyle scriptlevel="0">
    <mspace width="2em"></mspace>
  </mstyle>
  <mstyle scriptlevel="0">
    <mspace width="2em"></mspace>
  </mstyle>
  <msub>
    <mi mathvariant="bold">v</mi>
    <mi mathvariant="bold">i</mi>
  </msub>
  <mo>&#x2208;</mo>
  <mo fence="false" stretchy="false">{</mo>
  <mo mathvariant="bold">&#x2212;</mo>
  <mn mathvariant="bold">1</mn>
  <mo mathvariant="bold">,</mo>
  <mo mathvariant="bold">+</mo>
  <mn mathvariant="bold">1</mn>
  <mo fence="false" stretchy="false">}</mo>
  <mtext mathvariant="bold">&#xA0;or&#xA0;</mtext>
  <mo fence="false" stretchy="false">{</mo>
  <mn mathvariant="bold">0</mn>
  <mo mathvariant="bold">,</mo>
  <mn mathvariant="bold">1</mn>
  <mo fence="false" stretchy="false">}</mo>
</math>

其中abc是实数

我们需要定义一个代表我们问题的二次模型（QM）。构建一个二次二元模型（BQM）的最简单方式是使用dimod的符号变量。对于BQM中的每个数学变量，我们使用dimod定义一个符号二元变量。一旦QM被定义，它将被存储为一个BinaryQuadraticModel对象。这个对象存储了数学表达式的线性和二次系数，任何常数项或偏移量，以及用于构建模型的变量类型。在这种情况下，我们构建的bqmmodel对象会显示以下内容：

## 随机不全相等3-可满足性问题（NAE3SAT）

不全相等3-可满足性（NAE3SAT）是一个NP完全问题类，它的任务是满足若干个包含三个文字（变量或它们的否定）的合取子句。对于有效的解，每个子句中的文字应该是不全相等的；即除了(+1, +1, +1)或(-1, -1, -1)之外的任何值分配都对每个子句都是有效的。

每个子句在满足时对能量贡献-1，在不满足时贡献+3。因此，对于一个自旋分配s，它的能量H(s)下限为E(SAT) = -子句数，这个下限与可满足实例中的基态能量匹配。违反子句的数量是(H(s)-E(SAT))/4。

In [18]:
num_variables = 5;
rho = 2.1;
bqmodel = dimod.generators.random_nae3sat(num_variables, round(num_variables*rho))
print(bqmodel)


BinaryQuadraticModel({0: 0.0, 1: 0.0, 2: 0.0, 3: 0.0, 4: 0.0}, {(1, 0): 1.0, (2, 0): -1.0, (2, 1): 0.0, (3, 0): -3.0, (3, 1): 2.0, (3, 2): 1.0, (4, 0): -3.0, (4, 1): -1.0, (4, 2): 0.0, (4, 3): 2.0}, 0.0, 'SPIN')


基于我们的BQM定义Dynex BQM模型：

In [19]:
bqmodel.variables

Variables([0, 1, 2, 3, 4])

In [20]:
model = dynex.BQM(bqmodel);

## 与Dynex采样器互动

为了找到QM的最小能量状态（即为我们的QM提供最小能量值的变量值分配），Dynex SDK提供了采样器和求解器。求解器是运行问题的资源。采样器是多次运行问题以获取样本集合的过程，每个样本都是我们问题的一个可能解决方案。为方便起见，通常我们会将Dynex的采样器总称，也包括求解器。

In [21]:
sampler = dynex.DynexSampler(model);

一旦我们在程序中建立了我们的采样器，我们可以调用它来对我们的QM进行采样。每种类型的QM模型都有其自己的与采样器交互的方法，无论是QUBO、BinaryQuadrticModel，还是其他任何QM。我们使用Dynex的采样函数之一来调用采样器来对我们的QM进行采样，具体取决于我们使用的QM类型。例如，下面的代码片段演示了如何使用Dynex平台对名为bqm的BinaryQuadraticModel对象进行采样。

In [24]:
sampleset = sampler.sample(num_reads=32, annealing_time = 100);

time: 0.00s #workers: 2 #chips: 64 #steps: 110000 global loc: 3 global energy: 4.535805
FINISHED READ AFTER 0.00 SECONDS
SAMPLESET LOADED


在我们对QM进行采样后，采样器会返回一个SampleSet对象。这个对象包含了所有返回的样本及其相应的能量值、芯片数量、积分步数等信息。附加信息的内容取决于使用的采样器。随着用户对Dynex SDK和各种可用的采样器变得更加熟悉，花一些时间来探索SampleSet对象中提供的丰富信息通常是有用的。我们可以访问的SampleSet对象的一些关键属性和方法如下：

In [25]:
print(sampleset[-1]) # 能量最低的样本

{'sample': ['0.502819', '-0.985740', '0.006763', '-0.999880', '-0.528928', '0.000000', '0.000000', '-1.000000', '1.000000', '1.000000', '-1.000000', '0.000000', '0.000000', '-1.000000', '1.000000', '1.000000', '-1.000000', '-1.000000', '1.000000', '1.000000', '-1.000000', '1.000000'], 'chips': 64, 'steps': 110000, 'loc': 3, 'energy': 4.535805}


样本显示了我们的QM的相应能量值以及额外信息，如总能量、芯片数量或积分步数。

In [26]:
print(sampleset[-1]['sample'][:5]) # 前几个变量的能量水平:

['0.502819', '-0.985740', '0.006763', '-0.999880', '-0.528928']
